In [ ]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import pandas as pd
from geopy.geocoders import Bing

In [ ]:
data_aptos = pd.read_excel('data_aptos_final.xlsx')

### Nominatim (OSM)

In [ ]:
locator = Nominatim(user_agent = "puc_tcc")
lat_list = []
lon_list = []
cont = 0
for rua_num in data_aptos['Rua_Numero_Busca']:
    address = str(rua_num + ", boa viagem, recife").lower()
    location = locator.geocode(address)
    if location != None:
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
        print(location.raw['importance'])
    else:
        lat_list.append('NONE')
        lon_list.append('NONE')
        print('NONE!')
    
    cont = cont + 1
    print(cont)

            
data_aptos['lat'] = lat_list
data_aptos['lon'] = lon_list

### BING

In [ ]:
locator = Bing(api_key='Ar5fAeTZdy50Yq9qoi0UnOvmMeN0VUOc6dJ_ldOYcJ0PW8dZ-TIrdXJ6q9NJmHvM')
lat_list = []
lon_list = []
cont = 0
for rua_num in data_aptos['Rua_Numero_Busca']:
    address = str(rua_num + ", boa viagem, recife").lower()
    location = locator.geocode(address)
    if location != None:
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
    else:
        lat_list.append('NONE')
        lon_list.append('NONE')
        print('none!')
    
    cont = cont + 1
    print(cont)
            
data_aptos['lat_bing'] = lat_list
data_aptos['lon_bing'] = lon_list

### DISTÂNCIAS À ORLA

In [ ]:
def min_distance_orla(orla_df, ponto):
    distances = []
    for index, row in orla_df.iterrows():
        lat_orla = float(row['lat'])
        lon_orla = float(row['lon'])        
        distances.append(geodesic(ponto, (lat_orla, lon_orla)).m)
    
    return min(distances)
    
def min_distance_orla_bing(orla_df, ponto):
    distances = []
    for index, row in orla_df.iterrows():
        lat_orla = float(row['lat_bing'])
        lon_orla = float(row['lon_bing'])        
        distances.append(geodesic(ponto, (lat_orla, lon_orla)).m)
    
    return min(distances)
    

In [ ]:
orla_df = pd.read_excel('esig_enderecos_bv_pina.xlsx')
distances_list = []
distances_list_bing = []

for index, row in data_aptos.iterrows():
    # Nominatim --------------
    if row['lat'] != 'NONE':
        ponto = (float(row['lat']), float(row['lon']))
        distance = min_distance_orla(orla_df, ponto)
    else:
        distance = -99999
    distances_list.append(distance)
    
    # Bing --------------
    if row['lat_bing'] != 'NONE':
        ponto = (float(row['lat_bing']), float(row['lon_bing']))
        distance = min_distance_orla_bing(orla_df, ponto)
    else:
        distance = -99999
    distances_list_bing.append(distance)
    
    print(index)
    
    
data_aptos['distancia_orla'] = distances_list
data_aptos['distancia_orla_bing'] = distances_list_bing

In [ ]:
with pd.ExcelWriter("data_aptos_final.xlsx") as writer:
    data_aptos.to_excel(writer)  

In [ ]:
data_aptos.loc[data_aptos['distancia_orla_bing'] < 100 , ['Rua_Numero_Busca', 'distancia_orla']][100:150]

In [ ]:
len(data_aptos[data_aptos['lat'] == 'NONE'])

### Coordenadas da orla - Av. Boa Viagem - (EXECUTADO) - Nominatim

In [ ]:
orla_df = pd.read_excel('esig_enderecos_bv_pina.xlsx')

locator = Nominatim(user_agent = "puc_tcc")

lat_list = []
lon_list = []
cont = 0
for rua_num in orla_df['address']:
    address = rua_num.lower()
    location = locator.geocode(address)
    if location != None:
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
    else:
        lat_list.append('NONE')
        lon_list.append('NONE')
    
    cont = cont + 1
    print(cont)
            
orla_df['lat'] = lat_list
orla_df['lon'] = lon_list


### Coordenadas da orla - Av. Boa Viagem - (EXECUTADO) - Bing

In [ ]:
orla_df = pd.read_excel('esig_enderecos_bv_pina.xlsx')

locator = Bing(api_key='Ar5fAeTZdy50Yq9qoi0UnOvmMeN0VUOc6dJ_ldOYcJ0PW8dZ-TIrdXJ6q9NJmHvM')

lat_list = []
lon_list = []
cont = 0
for rua_num in orla_df['address']:
    address = rua_num.lower()
    location = locator.geocode(address)
    if location != None:
        lat_list.append(location.latitude)
        lon_list.append(location.longitude)
    else:
        lat_list.append('NONE')
        lon_list.append('NONE')
    
    cont = cont + 1
    print(cont)
            
orla_df['lat_bing'] = lat_list
orla_df['lon_bing'] = lon_list

Correções de coordenadas com ruas não encontradas no BING ou OSM

In [ ]:
orla_df = pd.read_excel('esig_enderecos_bv_pina.xlsx')

data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA JOAO ARRUDA MARINHO 54', ['lat', 'lon']] = ['-8.1065486603965', '-34.892144521043505']
data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA JOAO ARRUDA MARINHO 54', ['lat_bing', 'lon_bing']] = ['-8.1065486603965', '-34.892144521043505']
ponto = (-8.1065486603965, -34.892144521043505)
data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA JOAO ARRUDA MARINHO 54', ['distancia_orla', 'distancia_orla_bing']] = min_distance_orla(orla_df, ponto)

data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA AMARO ALBINO PIMENTEL 145', ['lat_bing', 'lon_bing']] = ['-8.108886850539902', '-34.897714682488655']
data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA AMARO ALBINO PIMENTEL 145', ['distancia_orla_bing']] = min_distance_orla_bing(orla_df, (-8.108886850539902, -34.897714682488655))

data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA PROFESSOR AURELIO DE CASTRO CAVALCANTI 79', ['lat_bing', 'lon_bing']] = ['-8.141487280000442', '-34.9126468040534']
data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA PROFESSOR AURELIO DE CASTRO CAVALCANTI 79', ['distancia_orla_bing']] = min_distance_orla_bing(orla_df, (-8.141487280000442, -34.9126468040534))

In [ ]:
data_aptos.loc[data_aptos['Rua_Numero_Busca'] == 'RUA AMARO ALBINO PIMENTEL 145']

In [ ]:
with pd.ExcelWriter("data_aptos_final.xlsx") as writer:
    data_aptos.to_excel(writer)  

### TESTES ISOLADOS - RAW DATA

In [16]:
from geopy.geocoders import Nominatim

endereco = 'Rua Marques de Valenca, 150, BOA VIAGEM, RECIFE'

locator = Nominatim(user_agent = "puc_tcc")
location = locator.geocode(endereco)

print('lat:', location.latitude)
print('lon:', location.longitude)


lat: -8.1288454
lon: -34.9042203


In [ ]:
endereco = 'RUA ANTONIO FALCAO NO 979, BOA VIAGEM, RECIFE'.lower()

locator = Nominatim(user_agent = "puc_tcc")
location = locator.geocode(endereco)
print(location.raw['class'])
print(location.raw['importance'])
display(location.raw)

locator = Bing(api_key='Ar5fAeTZdy50Yq9qoi0UnOvmMeN0VUOc6dJ_ldOYcJ0PW8dZ-TIrdXJ6q9NJmHvM')
location = locator.geocode(endereco)
display(location.raw)
print(location.raw['confidence'])